In [83]:
import unittest
import pandas as pd
import os, sys, io
from io import StringIO
import pkg_resources
import importlib
from colorama import Fore, Back, Style 
import ast
from datetime import datetime #, date, timedelta

<h1>Link Code to be Tested</h1>

In [2]:
# Reloads the devanalyst modules used in this notebook. Useful to refresh after code changes in the devanalyst modules
def reload():
    global timecard
    timecard = importlib.import_module('devanalyst.simulation.GenerateTimecards')
    timecard = importlib.reload(timecard)

In [85]:
# Re-run this each time the source code changes, so that we have the latest version
reload()

importing Jupyter notebook from c:\alex\code\labs\devanalyst\devanalyst\simulation\GenerateTimecards.ipynb


In [4]:
import devanalyst.simulation.GenerateTimecards as timecard
from devanalyst.simulation.GenerateTimecards import UserStoriesRepo, UserStory, UserStoryStatus, \
Ticket, TicketsRepo, ScrumTeam, ScrumTeamsRepo, IdCounter, WorkAssignments

<h1>Common Context Across Tests</h1>

<h2>Immutable Globals</h2>

In [5]:
#globals used in this test. After they are initialized in 'loadTestResources()', they don't change.
PM_DF = None
DEV_DF = None

<h2>Mutable Globals</h2>

In [6]:
# Global variable used to have a single counter for user stories id as they get generated in multiple calls. 
# Mutable state
NEXT_USER_STORY_ID = None

# Dictioaries of test results. Each key is the string name of a test and the value is the output of that test,
# expected or actual
EXPECTED = {}
ACTUAL = {}

<h2>Test Utilities</h2>

In [7]:
def initTestData(developers_df, productManagers_df, releaseDuration, sprintDuration=10):
# Test the code for creating user stories by constructing them and arranging them in a dataframe to display and evidence
# visually that the code works as it should

    global NEXT_USER_STORY_ID
    storiesRepo = UserStoriesRepo([])
    teamsRepo = ScrumTeamsRepo([])
    ticketsRepo = TicketsRepo([])
    NEXT_USER_STORY_ID = IdCounter()
    #cols = ['User Story Id','Scrum Team', 'Product Manager', 'Developer', 'Estimate',]
    userStoryId_vals = []
    scrumTeam_vals = []
    developer_vals = []
    productManager_vals = []
    estimate_vals = []
    
    teams_df = timecard.createTeamsDF(developers_df, productManagers_df)
    
    for team in teams_df['Scrum Team']:
        stories, backlog = timecard.createUserStoryBacklog(team, releaseDuration, sprintDuration, NEXT_USER_STORY_ID)
        storiesRepo.stories.extend(stories)
        teamsRepo.teams.append(team)
        for story in stories:
            scrumTeam_vals.append(team.teamId)
            userStoryId_vals.append(story.userStoryId)
            developer_vals.append(story.developer)
            productManager_vals.append(story.productManager)
            estimate_vals.append(story.originalEstimate)
        team.backlog = backlog
    stories_dict = {'User Story Id': userStoryId_vals, 'Scrum Team': scrumTeam_vals, 'Product Manager':productManager_vals, \
                'Developer': developer_vals, 'Estimate': estimate_vals}
    return teams_df, pd.DataFrame(stories_dict), teamsRepo, storiesRepo, ticketsRepo

In [8]:
def loadTestResources():
    global PM_DF
    global DEV_DF
    resource_path = '/'.join(('Resources', 'Simulation', 'Team.xlsx'))
    resource_package = 'devanalyst'
    fullpath = pkg_resources.resource_filename(resource_package, resource_path)
    PM_DF = pd.read_excel(fullpath, 'PMs')
    DEV_DF = pd.read_excel(fullpath, 'Dev')

loadTestResources()

In [9]:
def createExpectedOutput(expected_df, testName):
    resource_path = '/'.join(('Resources', 'Tests','Simulation', testName + '_EXPECTED.csv'))
    resource_package = 'devanalyst'
    fullpath = pkg_resources.resource_filename(resource_package, resource_path)
    return expected_df.to_csv(fullpath)

def loadExpectedOutput(testName, literal_cols=[]):
    resource_path = '/'.join(('Resources', 'Tests','Simulation', testName + '_EXPECTED.csv'))
    resource_package = 'devanalyst'
    fullpath = pkg_resources.resource_filename(resource_package, resource_path)
    df = pd.read_csv(fullpath)
    
    spurious_col = 'Unnamed: 0'
    if spurious_col in df.columns:
        df = df.drop([spurious_col], axis = 'columns') # Remove spurious index column, if any
    
    # Now for each column in list_cols, we have to treat it like it is a list so replace the contents of that
    # column with a parsing of the string value to to a list value
    for col in literal_cols:
        df[col] = df[col].apply(lambda x: ast.literal_eval(str(x)))
    return df

In [10]:
def matches(actual, expected):
    if isinstance(actual, str):
        return actual==expected
    else:
        return actual.equals(expected)

In [11]:
def testOK(testname):
    return matches(ACTUAL[testname], EXPECTED[testname])

<h1>test_uss</h1>
<p>Test User Story Status</p>

In [12]:
# Implement test logic, and run

# Helper methods to the test
def format_ticket(ticket):
    return ('Ticket:' + ticket.ticketId \
            + ',\n\t\t storyId=' + ticket.userStoryId \
            + ',\n\t\t costToFix=' + str(ticket.costToFix) \
            + ',\n\t\t effortToDate=' + str(ticket.effortToDate) \
            + ',\n\t\t percentAchieved=' + str(ticket.percentAchieved))
                
def format_tickets(tickets):
    output = ''
    for ticket in tickets:
        output = output + '\n\t\t{' + format_ticket(ticket) + '}'
    return output
            
def format_uss(uss):
    return ('\n *** USS:' + uss.userStoryId \
            + '\n\t achieved=' + str(uss.percentAchieved) \
            + ',\n\t planned=' + str(uss.planned) \
            + ',\n\t sprintPlanned=' + str(uss.sprintPlanned) \
            + ',\n\t tickets=' + format_tickets(uss.pendingTickets))

def format_item(item, item_label, sprint, timeInSprint):
    return ('\n *** ' + item_label + ' at ' + timeInSprint + ' of sprint' + str(sprint) + ': ' \
            + '\n\t userStoryId=' + str(item.userStoryId) \
            + ',\n\t taskType=' + str(item.taskType) \
            + ',\n\t ticketId=' + str(item.ticketId) \
            + ',\n\t estimate=' + '{0:.2f}'.format(item.estimate) \
            + ',\n\t percentAchieved=' + str(item.percentAchieved)  \
            + ',\n\t sprintPlanned=' + str(item.sprintPlanned))

# Test logic
def test_uss():
    output = '' 
    repo = UserStoriesRepo([UserStory('Story A', 25, 'Joe Developer', 'Amy PM'), \
                            UserStory('Story B', 17, 'Alex Developer', 'Kate PM')])
    uss = UserStoryStatus('Story B', 0.0)
    uss.planned = True
    uss.sprintPlanned = 1
    output = output + (format_uss(uss))
    item = uss.generateWorkItems(repo)[0]
    output = output + (format_item(item, 'Item#1', 1, 'start'))
    item.percentAchieved = 0.7
    newTickets = [Ticket('Bug 100','Story B', 4), Ticket('Bug 101','Story B', 1.5)]
    bugRepo = TicketsRepo(newTickets)
    output = output + (format_item(item, 'Item#1', 1, 'end'))
    uss.updateStatus([item], newTickets, bugRepo)
    uss.sprintPlanned = 2
    output = output + (format_uss(uss))

    items = uss.generateWorkItems(repo)
    item=items[0]
    output = output + (format_item(item, 'Item#1', 2, 'start'))
    item=items[1]
    output = output + (format_item(item, 'Item#2', 2, 'start'))
    item=items[2]
    output = output + (format_item(item, 'Item#3', 2, 'start'))
    items[0].percentAchieved = 0.9
    items[1].percentAchieved = 1.0
    items[2].percentAchieved = 0.5
    item=items[0]
    item=items[0]
    item=items[1]
    output = output + (format_item(item, 'Item#2', 2, 'end'))
    item=items[2]
    output = output + (format_item(item, 'Item#3', 2, 'end'))
    uss.updateStatus(items, [], bugRepo)
    output = output + (format_uss(uss))
    return output

# Run the test
test_uss_ACTUAL = test_uss()

In [13]:
# Uncomment to print a string output one can copy and paste into test_uss_EXPECTED
#test_uss_ACTUAL

In [14]:
# Set expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_uss_EXPECTED = '\n *** USS:Story B\n\t achieved=0.0,\n\t planned=True,\n\t sprintPlanned=1,\n\t tickets=\n *** Item#1 at start of sprint1: \n\t userStoryId=Story B,\n\t taskType=UNFINISHED_STORIES,\n\t ticketId=None,\n\t estimate=17.00,\n\t percentAchieved=0.0,\n\t sprintPlanned=1\n *** Item#1 at end of sprint1: \n\t userStoryId=Story B,\n\t taskType=UNFINISHED_STORIES,\n\t ticketId=None,\n\t estimate=17.00,\n\t percentAchieved=0.7,\n\t sprintPlanned=1\n *** USS:Story B\n\t achieved=0.7,\n\t planned=True,\n\t sprintPlanned=2,\n\t tickets=\n\t\t{Ticket:Bug 100,\n\t\t storyId=Story B,\n\t\t costToFix=4,\n\t\t effortToDate=0.0,\n\t\t percentAchieved=0.0}\n\t\t{Ticket:Bug 101,\n\t\t storyId=Story B,\n\t\t costToFix=1.5,\n\t\t effortToDate=0.0,\n\t\t percentAchieved=0.0}\n *** Item#1 at start of sprint2: \n\t userStoryId=Story B,\n\t taskType=UNFINISHED_STORIES,\n\t ticketId=None,\n\t estimate=5.10,\n\t percentAchieved=0.0,\n\t sprintPlanned=2\n *** Item#2 at start of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 100,\n\t estimate=2.00,\n\t percentAchieved=0.0,\n\t sprintPlanned=2\n *** Item#3 at start of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 101,\n\t estimate=2.00,\n\t percentAchieved=0.0,\n\t sprintPlanned=2\n *** Item#2 at end of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 100,\n\t estimate=2.00,\n\t percentAchieved=1.0,\n\t sprintPlanned=2\n *** Item#3 at end of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 101,\n\t estimate=2.00,\n\t percentAchieved=0.5,\n\t sprintPlanned=2\n *** USS:Story B\n\t achieved=0.97,\n\t planned=True,\n\t sprintPlanned=2,\n\t tickets=\n\t\t{Ticket:Bug 101,\n\t\t storyId=Story B,\n\t\t costToFix=1.5,\n\t\t effortToDate=2.0,\n\t\t percentAchieved=0.5}'

EXPECTED['uss'] = test_uss_EXPECTED
ACTUAL['uss'] = test_uss_ACTUAL
testOK('uss')

True

In [15]:
# Print ACTUAL output
print(Back.BLUE + Fore.WHITE + '--------------------- ACTUAL -------------------------', \
      Back.RESET + Fore.BLUE + '\n' + test_uss_ACTUAL) 

--------------------- ACTUAL ------------------------- 

 *** USS:Story B
	 achieved=0.0,
	 planned=True,
	 sprintPlanned=1,
	 tickets=
 *** Item#1 at start of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.0,
	 sprintPlanned=1
 *** Item#1 at end of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.7,
	 sprintPlanned=1
 *** USS:Story B
	 achieved=0.7,
	 planned=True,
	 sprintPlanned=2,
	 tickets=
		{Ticket:Bug 100,
		 storyId=Story B,
		 costToFix=4,
		 effortToDate=0.0,
		 percentAchieved=0.0}
		{Ticket:Bug 101,
		 storyId=Story B,
		 costToFix=1.5,
		 effortToDate=0.0,
		 percentAchieved=0.0}
 *** Item#1 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=5.10,
	 percentAchieved=0.0,
	 sprintPlanned=2
 *** Item#2 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=BUGS_ON_UNFINISHED_STORIES

In [16]:
# Print EXPECTED output
print(Back.GREEN + Fore.WHITE + '--------------------- EXPECTED -----------------------', \
      Back.RESET + Fore.GREEN + '\n' + test_uss_EXPECTED) 

--------------------- EXPECTED ----------------------- 

 *** USS:Story B
	 achieved=0.0,
	 planned=True,
	 sprintPlanned=1,
	 tickets=
 *** Item#1 at start of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.0,
	 sprintPlanned=1
 *** Item#1 at end of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.7,
	 sprintPlanned=1
 *** USS:Story B
	 achieved=0.7,
	 planned=True,
	 sprintPlanned=2,
	 tickets=
		{Ticket:Bug 100,
		 storyId=Story B,
		 costToFix=4,
		 effortToDate=0.0,
		 percentAchieved=0.0}
		{Ticket:Bug 101,
		 storyId=Story B,
		 costToFix=1.5,
		 effortToDate=0.0,
		 percentAchieved=0.0}
 *** Item#1 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=5.10,
	 percentAchieved=0.0,
	 sprintPlanned=2
 *** Item#2 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=BUGS_ON_UNFINISHED_STORIES

<h1>test_createTeams</h1>

In [17]:
# Implement test logic, and run

teams_df = timecard.createTeamsDF(DEV_DF, PM_DF)

object_cols = ['Scrum Team'] # Need to drop these since they print always-chaning memory address + can't save it in EXPECTED
test_createTeams_ACTUAL = teams_df.drop(object_cols, axis='columns')

In [18]:
# Uncomment to update expected output to match the actual one
#createExpectedOutput(test_createTeams_ACTUAL, 'test_createTeams')

In [19]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
list_cols = ['Developers', 'Product Managers', 'Areas of Responsibility']
test_createTeams_EXPECTED = loadExpectedOutput('test_createTeams', list_cols)

EXPECTED['createTeams'] = test_createTeams_EXPECTED
ACTUAL['createTeams'] = test_createTeams_ACTUAL
testOK('createTeams')

True

In [20]:
test_createTeams_ACTUAL

,Team Id,Developers,Product Managers,Areas of Responsibility
0,Team A,"[Anton Easterday, Beau Hockensmith, Bruno Stud...","[Sherlyn Cordle, Edgar Hibbler]","[Doctor, Patient]"
1,Team B,"[Hyun Jaffe, Isaura Casterline, Jacinto Immel,...",[Spencer Venezia],[Ministry of Health]
2,Team C,"[Lonnie Belz, Lorriane Demmer, Margorie Bering...",[Jamie Addington],[Hospital Administration]
3,Team D,"[Mohammad Tineo, Nohemi Santini, Olevia Haymak...",[Georgine Roan],[Insurance]


In [21]:
test_createTeams_EXPECTED

,Team Id,Developers,Product Managers,Areas of Responsibility
0,Team A,"[Anton Easterday, Beau Hockensmith, Bruno Stud...","[Sherlyn Cordle, Edgar Hibbler]","[Doctor, Patient]"
1,Team B,"[Hyun Jaffe, Isaura Casterline, Jacinto Immel,...",[Spencer Venezia],[Ministry of Health]
2,Team C,"[Lonnie Belz, Lorriane Demmer, Margorie Bering...",[Jamie Addington],[Hospital Administration]
3,Team D,"[Mohammad Tineo, Nohemi Santini, Olevia Haymak...",[Georgine Roan],[Insurance]


<h1>test_userStoryCreate</h1>
<p>This test has multiple views, each of which is checked separately</p>
<li>test_userStoryCreate_stories
<li>test_userStoryCreate_estimates
<li>test_userStoryCreate_crossCheck
<li>test_userStoryCreate_workload

In [22]:
# Implement test logic, and run it

#Test logic
def test_userStoryCreate():
    output = {}
    
    timecard.RANDOM.reset(271) # Set seed so output is the same even if logic invokes random methods
    
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = initTestData(DEV_DF, PM_DF, 125)
    
    grouped_estimates_df = stories_df.groupby([ 'Scrum Team', 'Developer'])['Estimate'].sum()
    workload_df = stories_df.groupby([ 'Scrum Team'])['User Story Id'].count()
    #workload_count_df = workload_df.count()
    
    avg_estimates_df = grouped_estimates_df.unstack().apply(lambda x: x.mean(), axis='columns')

    # Reset index to match the way how EXPECTED will be saved as a CSV file
    estimates_df      = grouped_estimates_df.reset_index()
    workload_df       = workload_df.reset_index()
    avg_estimates_df= avg_estimates_df.reset_index()
    
    # The unstacking above created an column with a number as the column name, 0. That will not match once expected output is saved
    # and reloaded, as it will come back as the string '0'. So rename that column to avoid spurious test failures
    avg_estimates_df = avg_estimates_df.rename(index=str, columns={0: 'Avg'})
    
    # Because of the manipulations, the index has changed and that will cause mistaches with the EXPECTED loaded from 
    # CSV. So re-index
    avg_estimates_df.index = pd.RangeIndex(start=0, stop=avg_estimates_df.index.size, step=1)
    
    crosscheck = [len(teams_df['Scrum Team'][0].backlog.pendingUserStories), 
                  len(teams_df['Scrum Team'][1].backlog.pendingUserStories),
                  len(teams_df['Scrum Team'][2].backlog.pendingUserStories), 
                  len(teams_df['Scrum Team'][3].backlog.pendingUserStories)]
    crosscheck_df = pd.DataFrame({'Team idx': [0,1,2,3], 'Backlog size': crosscheck})
    
    output['stories_df'] = stories_df
    output['estimates_df'] = estimates_df
    output['workload_df'] = workload_df
    output['crosscheck_df'] = crosscheck_df
    output['avg_estimates_df'] = avg_estimates_df

    return output

# Run the test
test_userStoryCreate_ACTUAL = test_userStoryCreate()

In [23]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_userStoryCreate_EXPECTED():
    createExpectedOutput(test_userStoryCreate_ACTUAL['stories_df'],        'test_userStoryCreate.stories_df')
    createExpectedOutput(test_userStoryCreate_ACTUAL['estimates_df'],      'test_userStoryCreate.estimates_df')
    createExpectedOutput(test_userStoryCreate_ACTUAL['workload_df'],       'test_userStoryCreate.workload_df')
    createExpectedOutput(test_userStoryCreate_ACTUAL['crosscheck_df'],     'test_userStoryCreate.crosscheck_df')
    createExpectedOutput(test_userStoryCreate_ACTUAL['avg_estimates_df'],  'test_userStoryCreate.avg_estimates_df')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_userStoryCreate_EXPECTED()

In [24]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK

test_userStoryCreate_EXPECTED = {}

test_userStoryCreate_EXPECTED['stories_df']         = loadExpectedOutput('test_userStoryCreate.stories_df')
test_userStoryCreate_EXPECTED['estimates_df']       = loadExpectedOutput('test_userStoryCreate.estimates_df')
test_userStoryCreate_EXPECTED['workload_df']        = loadExpectedOutput('test_userStoryCreate.workload_df')
test_userStoryCreate_EXPECTED['crosscheck_df']      = loadExpectedOutput('test_userStoryCreate.crosscheck_df')
test_userStoryCreate_EXPECTED['avg_estimates_df']   = loadExpectedOutput('test_userStoryCreate.avg_estimates_df')

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for the sensitive fields
rounded = test_userStoryCreate_EXPECTED['avg_estimates_df']['Avg'].apply(lambda x: round(x, 6)) # Round to 6 decimal places
test_userStoryCreate_EXPECTED['avg_estimates_df']['Avg'] = rounded
rounded = test_userStoryCreate_ACTUAL['avg_estimates_df']['Avg'].apply(lambda x: round(x, 6)) # Round to 6 decimal places
test_userStoryCreate_ACTUAL['avg_estimates_df']['Avg'] = rounded

EXPECTED['test_userStoryCreate.stories_df']         = test_userStoryCreate_EXPECTED['stories_df']
EXPECTED['test_userStoryCreate.estimates_df']       = test_userStoryCreate_EXPECTED['estimates_df']
EXPECTED['test_userStoryCreate.workload_df']        = test_userStoryCreate_EXPECTED['workload_df']
EXPECTED['test_userStoryCreate.crosscheck_df']      = test_userStoryCreate_EXPECTED['crosscheck_df']
EXPECTED['test_userStoryCreate.avg_estimates_df']   = test_userStoryCreate_EXPECTED['avg_estimates_df']

ACTUAL['test_userStoryCreate.stories_df']           = test_userStoryCreate_ACTUAL['stories_df']
ACTUAL['test_userStoryCreate.estimates_df']         = test_userStoryCreate_ACTUAL['estimates_df']
ACTUAL['test_userStoryCreate.workload_df']          = test_userStoryCreate_ACTUAL['workload_df']
ACTUAL['test_userStoryCreate.crosscheck_df']        = test_userStoryCreate_ACTUAL['crosscheck_df']
ACTUAL['test_userStoryCreate.avg_estimates_df']     = test_userStoryCreate_ACTUAL['avg_estimates_df']

testOK('test_userStoryCreate.stories_df'), \
testOK('test_userStoryCreate.estimates_df'), \
testOK('test_userStoryCreate.workload_df'), \
testOK('test_userStoryCreate.crosscheck_df'), \
testOK('test_userStoryCreate.avg_estimates_df')

(True, True, True, True, True)

In [25]:
test_userStoryCreate_ACTUAL['stories_df']

,User Story Id,Scrum Team,Product Manager,Developer,Estimate
0,UserStory #1,Team A,Sherlyn Cordle,Beau Hockensmith,10
1,UserStory #2,Team A,Sherlyn Cordle,Glenna Mcghie,10
2,UserStory #3,Team A,Edgar Hibbler,Francisco Hoppe,4
3,UserStory #4,Team A,Sherlyn Cordle,Beau Hockensmith,7
4,UserStory #5,Team A,Edgar Hibbler,Gregorio Darr,3
5,UserStory #6,Team A,Sherlyn Cordle,Beau Hockensmith,3
6,UserStory #7,Team A,Sherlyn Cordle,Anton Easterday,8
7,UserStory #8,Team A,Edgar Hibbler,Anton Easterday,9
8,UserStory #9,Team A,Sherlyn Cordle,Gregorio Darr,9
9,UserStory #10,Team A,Sherlyn Cordle,Gregorio Darr,9


In [26]:
test_userStoryCreate_EXPECTED['stories_df']

,User Story Id,Scrum Team,Product Manager,Developer,Estimate
0,UserStory #1,Team A,Sherlyn Cordle,Beau Hockensmith,10
1,UserStory #2,Team A,Sherlyn Cordle,Glenna Mcghie,10
2,UserStory #3,Team A,Edgar Hibbler,Francisco Hoppe,4
3,UserStory #4,Team A,Sherlyn Cordle,Beau Hockensmith,7
4,UserStory #5,Team A,Edgar Hibbler,Gregorio Darr,3
5,UserStory #6,Team A,Sherlyn Cordle,Beau Hockensmith,3
6,UserStory #7,Team A,Sherlyn Cordle,Anton Easterday,8
7,UserStory #8,Team A,Edgar Hibbler,Anton Easterday,9
8,UserStory #9,Team A,Sherlyn Cordle,Gregorio Darr,9
9,UserStory #10,Team A,Sherlyn Cordle,Gregorio Darr,9


In [27]:
test_userStoryCreate_ACTUAL['estimates_df']

,Scrum Team,Developer,Estimate
0,Team A,Anton Easterday,117
1,Team A,Beau Hockensmith,125
2,Team A,Bruno Studley,122
3,Team A,Craig Garlitz,125
4,Team A,Francisco Hoppe,125
5,Team A,Glenna Mcghie,122
6,Team A,Gregorio Darr,120
7,Team A,Heriberto Martini,122
8,Team B,Hyun Jaffe,123
9,Team B,Isaura Casterline,114


In [28]:
test_userStoryCreate_EXPECTED['estimates_df']

,Scrum Team,Developer,Estimate
0,Team A,Anton Easterday,117
1,Team A,Beau Hockensmith,125
2,Team A,Bruno Studley,122
3,Team A,Craig Garlitz,125
4,Team A,Francisco Hoppe,125
5,Team A,Glenna Mcghie,122
6,Team A,Gregorio Darr,120
7,Team A,Heriberto Martini,122
8,Team B,Hyun Jaffe,123
9,Team B,Isaura Casterline,114


In [29]:
test_userStoryCreate_ACTUAL['workload_df']

,Scrum Team,User Story Id
0,Team A,185
1,Team B,125
2,Team C,144
3,Team D,205


In [30]:
test_userStoryCreate_EXPECTED['workload_df']

,Scrum Team,User Story Id
0,Team A,185
1,Team B,125
2,Team C,144
3,Team D,205


In [31]:
test_userStoryCreate_ACTUAL['crosscheck_df']

,Team idx,Backlog size
0,0,185
1,1,125
2,2,144
3,3,205


In [32]:
test_userStoryCreate_EXPECTED['crosscheck_df']

,Team idx,Backlog size
0,0,185
1,1,125
2,2,144
3,3,205


In [33]:
test_userStoryCreate_ACTUAL['avg_estimates_df']

,Scrum Team,Avg
0,Team A,122.250000
1,Team B,119.333333
2,Team C,121.571429
3,Team D,122.888889


In [34]:
test_userStoryCreate_EXPECTED['avg_estimates_df']

,Scrum Team,Avg
0,Team A,122.250000
1,Team B,119.333333
2,Team C,121.571429
3,Team D,122.888889


<h1>test_WorkAssignments</h1>

In [35]:
# Implement test logic, and run it

#Test logic
def test_WorkAssignments():
    output = {}
    timecard.RANDOM.reset(271) # Set seed so output is the same even if logic invokes random methods
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = initTestData(DEV_DF, PM_DF, 125)

    teamId0 = teams_df['Scrum Team'][0].teamId
    work = WorkAssignments(teamId0, teamsRepo, storiesRepo, sprint=1)
    initial_df = work.committedTime(10)
    # Test re-assigning of work
    item = work.allocations[timecard.UNPLANNED][timecard.OWNER_TBD][timecard.UNFINISHED_STORIES][25]
    work.reAssign(item, 'Bruno Studley', timecard.CURRENT_SPRINT)
    final_df = work.committedTime(10)
    
    output['Initial'] = initial_df
    output['Final'] = final_df

    return output
    
# Run the test
test_WorkAssignments_ACTUAL = test_WorkAssignments()

In [36]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_WorkAssignments_EXPECTED():
    createExpectedOutput(test_WorkAssignments_ACTUAL['Initial'],        'test_WorkAssignments.Initial')
    createExpectedOutput(test_WorkAssignments_ACTUAL['Final'],          'test_WorkAssignments.Final')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_WorkAssignments_EXPECTED()

In [37]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_WorkAssignments_EXPECTED = {}

test_WorkAssignments_EXPECTED['Initial']         = loadExpectedOutput('test_WorkAssignments.Initial')
test_WorkAssignments_EXPECTED['Final']       = loadExpectedOutput('test_WorkAssignments.Final')

EXPECTED['test_WorkAssignments.Initial']         = test_WorkAssignments_EXPECTED['Initial']
EXPECTED['test_WorkAssignments.Final']           = test_WorkAssignments_EXPECTED['Final']

ACTUAL['test_WorkAssignments.Initial']           = test_WorkAssignments_ACTUAL['Initial']
ACTUAL['test_WorkAssignments.Final']             = test_WorkAssignments_ACTUAL['Final']

testOK('test_WorkAssignments.Initial'), \
testOK('test_WorkAssignments.Final'), \


(True, True)

In [38]:
test_WorkAssignments_ACTUAL['Initial']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
1,Beau Hockensmith,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
2,Bruno Studley,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
3,Craig Garlitz,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
4,Francisco Hoppe,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
5,Glenna Mcghie,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
6,Gregorio Darr,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
7,Heriberto Martini,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
8,OWNER_TBD,0.0,0,0.0,0,978.0,185,NaN,0.0,0,0


In [39]:
test_WorkAssignments_EXPECTED['Initial']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
1,Beau Hockensmith,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
2,Bruno Studley,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
3,Craig Garlitz,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
4,Francisco Hoppe,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
5,Glenna Mcghie,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
6,Gregorio Darr,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
7,Heriberto Martini,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20
8,OWNER_TBD,0.0,0,0.0,0,978.0,185,NaN,0.0,0,0


In [40]:
test_WorkAssignments_ACTUAL['Final']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
1,Beau Hockensmith,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
2,Bruno Studley,0.0,0,0.0,0,7.0,1,3.0,0.0,0,13.0
3,Craig Garlitz,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
4,Francisco Hoppe,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
5,Glenna Mcghie,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
6,Gregorio Darr,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
7,Heriberto Martini,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
8,OWNER_TBD,0.0,0,0.0,0,971.0,184,NaN,0.0,0,0.0


In [41]:
test_WorkAssignments_EXPECTED['Final']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
1,Beau Hockensmith,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
2,Bruno Studley,0.0,0,0.0,0,7.0,1,3.0,0.0,0,13.0
3,Craig Garlitz,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
4,Francisco Hoppe,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
5,Glenna Mcghie,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
6,Gregorio Darr,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
7,Heriberto Martini,0.0,0,0.0,0,0.0,0,10.0,0.0,0,20.0
8,OWNER_TBD,0.0,0,0.0,0,971.0,184,NaN,0.0,0,0.0


<h1>test_oneSprint</h1>

In [42]:
# Implement test logic, and run it

#Test logic
def test_oneSprint():    
    output = {}
    timecard.RANDOM.reset(271) # Set seed so output is the same even if logic invokes random methods
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = initTestData(DEV_DF, PM_DF, 125)

    # Select a team
    teamId = teams_df['Scrum Team'][0].teamId
    
    # Choose what to work on at the start of a sprint.
    SPRINT_DURATION = 10
    work = timecard.chooseWhatToDoInSprint(teamId, teamsRepo, storiesRepo, SPRINT_DURATION, sprint=1)
    start_committed_df = work.committedTime(SPRINT_DURATION)
    start_tasks_df = work.committedTasks()
    
    # Deliver what the sprint actually accomplished, including ingest of defects arriving during sprint
    timecard.deliverSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, SPRINT_DURATION) # mutates work
    inflow = timecard.inflowOfTickets(teamId, teamsRepo, ticketsRepo, storiesRepo)
    end_committed_df = work.committedTime(0) # Sprint is over, so sprint capacity parameter is 0
    
    # Test continued
    timecard.updateBacklogAfterSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, inflow)
    end_tasks_df = work.committedTasks()
    
    output['Start_Committed'] = start_committed_df
    output['Start_Tasks'] = start_tasks_df
    output['End_Committed'] = end_committed_df
    output['End_Tasks'] = end_tasks_df
    return output

# Run the test
test_oneSprint_ACTUAL = test_oneSprint()

In [43]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_oneSprint_EXPECTED():
    createExpectedOutput(test_oneSprint_ACTUAL['Start_Committed'],    'test_oneSprint.Start_Committed')
    createExpectedOutput(test_oneSprint_ACTUAL['Start_Tasks'],        'test_oneSprint.Start_Tasks')
    createExpectedOutput(test_oneSprint_ACTUAL['End_Committed'],      'test_oneSprint.End_Committed')
    createExpectedOutput(test_oneSprint_ACTUAL['End_Tasks'],          'test_oneSprint.End_Tasks')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_oneSprint_EXPECTED()

In [44]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_oneSprint_EXPECTED = {}

test_oneSprint_EXPECTED['Start_Committed']      = loadExpectedOutput('test_oneSprint.Start_Committed')
test_oneSprint_EXPECTED['Start_Tasks']          = loadExpectedOutput('test_oneSprint.Start_Tasks')
test_oneSprint_EXPECTED['End_Committed']        = loadExpectedOutput('test_oneSprint.End_Committed')
test_oneSprint_EXPECTED['End_Tasks']            = loadExpectedOutput('test_oneSprint.End_Tasks')

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for sensitive fields (any float)
sensitive_fields = ['Rejects (days)', 'Debugging (days)', 'Implementation (days)', 'Bandwidth',\
                    'NEXT SPRINT (days)', 'NEXT SPRINT Bandwidth']
testlets = ['Start_Committed', 'End_Committed']
for field in sensitive_fields:
    for testlet in testlets:
        rounded = test_oneSprint_EXPECTED[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_oneSprint_EXPECTED[testlet][field] = rounded
        rounded = test_oneSprint_ACTUAL[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_oneSprint_ACTUAL[testlet][field] = rounded
sensitive_fields = ['Original Estimate', 'Effort Spent', 'Effort Remaining', 'Percent Achieved']
testlets = ['Start_Tasks', 'End_Tasks']
for field in sensitive_fields:
    for testlet in testlets:
        rounded = test_oneSprint_EXPECTED[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_oneSprint_EXPECTED[testlet][field] = rounded
        rounded = test_oneSprint_ACTUAL[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_oneSprint_ACTUAL[testlet][field] = rounded

EXPECTED['test_oneSprint.Start_Committed']      = test_oneSprint_EXPECTED['Start_Committed']
EXPECTED['test_oneSprint.Start_Tasks']          = test_oneSprint_EXPECTED['Start_Tasks']
EXPECTED['test_oneSprint.End_Committed']        = test_oneSprint_EXPECTED['End_Committed']
EXPECTED['test_oneSprint.End_Tasks']            = test_oneSprint_EXPECTED['End_Tasks']

ACTUAL['test_oneSprint.Start_Committed']        = test_oneSprint_ACTUAL['Start_Committed']
ACTUAL['test_oneSprint.Start_Tasks']            = test_oneSprint_ACTUAL['Start_Tasks']
ACTUAL['test_oneSprint.End_Committed']          = test_oneSprint_ACTUAL['End_Committed']
ACTUAL['test_oneSprint.End_Tasks']              = test_oneSprint_ACTUAL['End_Tasks']

testOK('test_oneSprint.Start_Committed'), \
testOK('test_oneSprint.Start_Tasks'), \
testOK('test_oneSprint.End_Committed'), \
testOK('test_oneSprint.End_Tasks'), \

(True, True, True, True)

In [45]:
test_oneSprint_ACTUAL['Start_Committed']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,10.0,4,0.0,0.0,0,10.0
1,Beau Hockensmith,0.0,0,0.0,0,10.0,1,0.0,0.0,0,10.0
2,Bruno Studley,0.0,0,0.0,0,10.0,3,0.0,0.0,0,10.0
3,Craig Garlitz,0.0,0,0.0,0,10.0,2,0.0,0.0,0,10.0
4,Francisco Hoppe,0.0,0,0.0,0,10.0,2,0.0,0.0,0,10.0
5,Glenna Mcghie,0.0,0,0.0,0,10.0,2,0.0,0.0,0,10.0
6,Gregorio Darr,0.0,0,0.0,0,10.0,3,0.0,0.0,0,10.0
7,Heriberto Martini,0.0,0,0.0,0,10.0,2,0.0,0.0,0,10.0
8,OWNER_TBD,0.0,0,0.0,0,898.0,166,NaN,0.0,0,0.0


In [46]:
test_oneSprint_EXPECTED['Start_Committed']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,10.0,4,0.0,0.0,0,10.0
1,Beau Hockensmith,0.0,0,0.0,0,10.0,1,0.0,0.0,0,10.0
2,Bruno Studley,0.0,0,0.0,0,10.0,3,0.0,0.0,0,10.0
3,Craig Garlitz,0.0,0,0.0,0,10.0,2,0.0,0.0,0,10.0
4,Francisco Hoppe,0.0,0,0.0,0,10.0,2,0.0,0.0,0,10.0
5,Glenna Mcghie,0.0,0,0.0,0,10.0,2,0.0,0.0,0,10.0
6,Gregorio Darr,0.0,0,0.0,0,10.0,3,0.0,0.0,0,10.0
7,Heriberto Martini,0.0,0,0.0,0,10.0,2,0.0,0.0,0,10.0
8,OWNER_TBD,0.0,0,0.0,0,898.0,166,NaN,0.0,0,0.0


In [47]:
test_oneSprint_ACTUAL['Start_Tasks']

,Owner,Task Type,User Story Id,Planned for Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved
0,Anton Easterday,UNFINISHED_STORIES,UserStory #151,1,2.0,CURRENT_SPRINT,0.0,2.0,0.0
1,Anton Easterday,UNFINISHED_STORIES,UserStory #97,1,2.0,CURRENT_SPRINT,0.0,2.0,0.0
2,Anton Easterday,UNFINISHED_STORIES,UserStory #58,1,4.0,CURRENT_SPRINT,0.0,4.0,0.0
3,Anton Easterday,UNFINISHED_STORIES,UserStory #83,1,2.0,CURRENT_SPRINT,0.0,2.0,0.0
4,Beau Hockensmith,UNFINISHED_STORIES,UserStory #1,1,10.0,CURRENT_SPRINT,0.0,10.0,0.0
5,Bruno Studley,UNFINISHED_STORIES,UserStory #77,1,3.0,CURRENT_SPRINT,0.0,3.0,0.0
6,Bruno Studley,UNFINISHED_STORIES,UserStory #137,1,2.0,CURRENT_SPRINT,0.0,2.0,0.0
7,Bruno Studley,UNFINISHED_STORIES,UserStory #52,1,5.0,CURRENT_SPRINT,0.0,5.0,0.0
8,Craig Garlitz,UNFINISHED_STORIES,UserStory #143,1,9.0,CURRENT_SPRINT,0.0,9.0,0.0
9,Craig Garlitz,UNFINISHED_STORIES,UserStory #75,1,1.0,CURRENT_SPRINT,0.0,1.0,0.0


In [48]:
test_oneSprint_EXPECTED['Start_Tasks']

,Owner,Task Type,User Story Id,Planned for Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved
0,Anton Easterday,UNFINISHED_STORIES,UserStory #151,1,2.0,CURRENT_SPRINT,0.0,2.0,0.0
1,Anton Easterday,UNFINISHED_STORIES,UserStory #97,1,2.0,CURRENT_SPRINT,0.0,2.0,0.0
2,Anton Easterday,UNFINISHED_STORIES,UserStory #58,1,4.0,CURRENT_SPRINT,0.0,4.0,0.0
3,Anton Easterday,UNFINISHED_STORIES,UserStory #83,1,2.0,CURRENT_SPRINT,0.0,2.0,0.0
4,Beau Hockensmith,UNFINISHED_STORIES,UserStory #1,1,10.0,CURRENT_SPRINT,0.0,10.0,0.0
5,Bruno Studley,UNFINISHED_STORIES,UserStory #77,1,3.0,CURRENT_SPRINT,0.0,3.0,0.0
6,Bruno Studley,UNFINISHED_STORIES,UserStory #137,1,2.0,CURRENT_SPRINT,0.0,2.0,0.0
7,Bruno Studley,UNFINISHED_STORIES,UserStory #52,1,5.0,CURRENT_SPRINT,0.0,5.0,0.0
8,Craig Garlitz,UNFINISHED_STORIES,UserStory #143,1,9.0,CURRENT_SPRINT,0.0,9.0,0.0
9,Craig Garlitz,UNFINISHED_STORIES,UserStory #75,1,1.0,CURRENT_SPRINT,0.0,1.0,0.0


In [49]:
test_oneSprint_ACTUAL['End_Committed']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,2.0,4,-2.0,0.0,0,-2.0
1,Beau Hockensmith,0.0,0,0.0,0,2.0,1,-2.0,0.0,0,-2.0
2,Bruno Studley,0.0,0,0.0,0,2.0,3,-2.0,0.0,0,-2.0
3,Craig Garlitz,0.0,0,0.0,0,2.0,2,-2.0,0.0,0,-2.0
4,Francisco Hoppe,0.0,0,0.0,0,2.0,2,-2.0,0.0,0,-2.0
5,Glenna Mcghie,0.0,0,0.0,0,2.0,2,-2.0,0.0,0,-2.0
6,Gregorio Darr,0.0,0,0.0,0,2.0,3,-2.0,0.0,0,-2.0
7,Heriberto Martini,0.0,0,0.0,0,2.0,2,-2.0,0.0,0,-2.0
8,OWNER_TBD,0.0,0,0.0,0,898.0,166,NaN,0.0,0,0.0


In [50]:
test_oneSprint_EXPECTED['End_Committed']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,2.0,4,-2.0,0.0,0,-2.0
1,Beau Hockensmith,0.0,0,0.0,0,2.0,1,-2.0,0.0,0,-2.0
2,Bruno Studley,0.0,0,0.0,0,2.0,3,-2.0,0.0,0,-2.0
3,Craig Garlitz,0.0,0,0.0,0,2.0,2,-2.0,0.0,0,-2.0
4,Francisco Hoppe,0.0,0,0.0,0,2.0,2,-2.0,0.0,0,-2.0
5,Glenna Mcghie,0.0,0,0.0,0,2.0,2,-2.0,0.0,0,-2.0
6,Gregorio Darr,0.0,0,0.0,0,2.0,3,-2.0,0.0,0,-2.0
7,Heriberto Martini,0.0,0,0.0,0,2.0,2,-2.0,0.0,0,-2.0
8,OWNER_TBD,0.0,0,0.0,0,898.0,166,NaN,0.0,0,0.0


In [51]:
test_oneSprint_ACTUAL['End_Tasks']

,Owner,Task Type,User Story Id,Planned for Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved
0,Anton Easterday,UNFINISHED_STORIES,UserStory #151,1,2.0,CURRENT_SPRINT,2.50,0.0,1.000000
1,Anton Easterday,UNFINISHED_STORIES,UserStory #97,1,2.0,CURRENT_SPRINT,2.50,0.0,1.000000
2,Anton Easterday,UNFINISHED_STORIES,UserStory #58,1,4.0,CURRENT_SPRINT,5.00,0.0,1.000000
3,Anton Easterday,UNFINISHED_STORIES,UserStory #83,1,2.0,CURRENT_SPRINT,0.00,2.0,0.000000
4,Beau Hockensmith,UNFINISHED_STORIES,UserStory #1,1,10.0,CURRENT_SPRINT,10.00,2.0,0.800000
5,Bruno Studley,UNFINISHED_STORIES,UserStory #77,1,3.0,CURRENT_SPRINT,3.75,0.0,1.000000
6,Bruno Studley,UNFINISHED_STORIES,UserStory #137,1,2.0,CURRENT_SPRINT,2.50,0.0,1.000000
7,Bruno Studley,UNFINISHED_STORIES,UserStory #52,1,5.0,CURRENT_SPRINT,3.75,2.0,0.600000
8,Craig Garlitz,UNFINISHED_STORIES,UserStory #143,1,9.0,CURRENT_SPRINT,10.00,1.0,0.888889
9,Craig Garlitz,UNFINISHED_STORIES,UserStory #75,1,1.0,CURRENT_SPRINT,0.00,1.0,0.000000


In [52]:
test_oneSprint_EXPECTED['End_Tasks']

,Owner,Task Type,User Story Id,Planned for Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved
0,Anton Easterday,UNFINISHED_STORIES,UserStory #151,1,2.0,CURRENT_SPRINT,2.50,0.0,1.000000
1,Anton Easterday,UNFINISHED_STORIES,UserStory #97,1,2.0,CURRENT_SPRINT,2.50,0.0,1.000000
2,Anton Easterday,UNFINISHED_STORIES,UserStory #58,1,4.0,CURRENT_SPRINT,5.00,0.0,1.000000
3,Anton Easterday,UNFINISHED_STORIES,UserStory #83,1,2.0,CURRENT_SPRINT,0.00,2.0,0.000000
4,Beau Hockensmith,UNFINISHED_STORIES,UserStory #1,1,10.0,CURRENT_SPRINT,10.00,2.0,0.800000
5,Bruno Studley,UNFINISHED_STORIES,UserStory #77,1,3.0,CURRENT_SPRINT,3.75,0.0,1.000000
6,Bruno Studley,UNFINISHED_STORIES,UserStory #137,1,2.0,CURRENT_SPRINT,2.50,0.0,1.000000
7,Bruno Studley,UNFINISHED_STORIES,UserStory #52,1,5.0,CURRENT_SPRINT,3.75,2.0,0.600000
8,Craig Garlitz,UNFINISHED_STORIES,UserStory #143,1,9.0,CURRENT_SPRINT,10.00,1.0,0.888889
9,Craig Garlitz,UNFINISHED_STORIES,UserStory #75,1,1.0,CURRENT_SPRINT,0.00,1.0,0.000000


<h1>test_multipleSprints</h1>

In [53]:
# Implement test logic, and run it

#Test logic
def test_multipleSprints():
    output = {}
    timecard.RANDOM.reset(271) # Set seed so output is the same even if logic invokes random methods
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = initTestData(DEV_DF, PM_DF, 125)
    
    # Select a team
    teamId = teams_df['Scrum Team'][0].teamId
    
    # Test many sprints into the future, to see if eventually people have extra time and start using that extra time
    # in the current sprint to get a head start on tasks for the next sprint
    SPRINT_DURATION = 10
    NUMBER_OF_SPRINTS = 15 

    work = None
    for i in range(NUMBER_OF_SPRINTS):
        work = timecard.chooseWhatToDoInSprint(teamId, teamsRepo, storiesRepo, SPRINT_DURATION, sprint=i+1)
        if (i== NUMBER_OF_SPRINTS -1):
            break
        timecard.deliverSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, SPRINT_DURATION) # This mutates 'work'
        inflow = timecard.inflowOfTickets(teamId, teamsRepo, ticketsRepo, storiesRepo)
        timecard.updateBacklogAfterSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, inflow) # Does not mutate 'work'
    
    # Work Assignments at the start of the last sprint. Should see some "looking ahead" tasks, i.e., tasks that would
    # normally be done in the next sprint but are started in this sprint since we have time leftover from this sprint's 
    # deliverables
    last = work
    start_committed_df = last.committedTime(10)
    start_tasks_df     = last.committedTasks()
    
    # Finish this last sprint and confirm we spent time in some of the deliveries for the next sprint (i.e., that we
    # were "looking ahead")
    timecard.deliverSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, last, 10)
    end_committed_df = last.committedTime(0)
    end_tasks_df     =last.committedTasks()
    
    output['start_committed']   = start_committed_df
    output['start_tasks']       = start_tasks_df
    output['end_committed']     = end_committed_df
    output['end_tasks']         = end_tasks_df

    return output

# Run the test
test_multipleSprints_ACTUAL = test_multipleSprints()

In [54]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_multipleSprints_EXPECTED():
    createExpectedOutput(test_multipleSprints_ACTUAL['start_committed'],    'test_multipleSprints.start_committed')
    createExpectedOutput(test_multipleSprints_ACTUAL['start_tasks'],        'test_multipleSprints.start_tasks')
    createExpectedOutput(test_multipleSprints_ACTUAL['end_committed'],      'test_multipleSprints.end_committed')
    createExpectedOutput(test_multipleSprints_ACTUAL['end_tasks'],          'test_multipleSprints.end_tasks')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_multipleSprints_EXPECTED()

In [55]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_multipleSprints_EXPECTED = {}

test_multipleSprints_EXPECTED['start_committed']    = loadExpectedOutput('test_multipleSprints.start_committed')
test_multipleSprints_EXPECTED['start_tasks']        = loadExpectedOutput('test_multipleSprints.start_tasks')
test_multipleSprints_EXPECTED['end_committed']      = loadExpectedOutput('test_multipleSprints.end_committed')
test_multipleSprints_EXPECTED['end_tasks']          = loadExpectedOutput('test_multipleSprints.end_tasks')

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for sensitive fields (any float)
sensitive_fields = ['Rejects (days)', 'Debugging (days)', 'Implementation (days)', 'Bandwidth',\
                    'NEXT SPRINT (days)', 'NEXT SPRINT Bandwidth']
testlets = ['start_committed', 'end_committed']
for field in sensitive_fields:
    for testlet in testlets:
        rounded = test_multipleSprints_EXPECTED[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_multipleSprints_EXPECTED[testlet][field] = rounded
        rounded = test_multipleSprints_ACTUAL[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_multipleSprints_ACTUAL[testlet][field] = rounded
sensitive_fields = ['Original Estimate', 'Effort Spent', 'Effort Remaining', 'Percent Achieved']
testlets = ['start_tasks', 'end_tasks']
for field in sensitive_fields:
    for testlet in testlets:
        rounded = test_multipleSprints_EXPECTED[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_multipleSprints_EXPECTED[testlet][field] = rounded
        rounded = test_multipleSprints_ACTUAL[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_multipleSprints_ACTUAL[testlet][field] = rounded

EXPECTED['test_multipleSprints.start_committed']    = test_multipleSprints_EXPECTED['start_committed']
EXPECTED['test_multipleSprints.start_tasks']        = test_multipleSprints_EXPECTED['start_tasks']
EXPECTED['test_multipleSprints.end_committed']      = test_multipleSprints_EXPECTED['end_committed']
EXPECTED['test_multipleSprints.end_tasks']          = test_multipleSprints_EXPECTED['end_tasks']

ACTUAL['test_multipleSprints.start_committed']      = test_multipleSprints_ACTUAL['start_committed']
ACTUAL['test_multipleSprints.start_tasks']          = test_multipleSprints_ACTUAL['start_tasks']
ACTUAL['test_multipleSprints.end_committed']        = test_multipleSprints_ACTUAL['end_committed']
ACTUAL['test_multipleSprints.end_tasks']            = test_multipleSprints_ACTUAL['end_tasks']

testOK('test_multipleSprints.start_committed'), \
testOK('test_multipleSprints.start_tasks'), \
testOK('test_multipleSprints.end_committed'), \
testOK('test_multipleSprints.end_tasks'), \

(True, True, True, True)

In [56]:
test_multipleSprints_ACTUAL['start_committed']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,3.0,2,7.0,9.0,1,8.0
1,Beau Hockensmith,0.0,0,0.0,0,3.0,1,7.0,9.0,1,8.0
2,Bruno Studley,0.0,0,0.0,0,2.0,2,8.0,0.0,0,18.0
3,Craig Garlitz,0.0,0,0.0,0,3.0,1,7.0,10.0,1,7.0
4,Francisco Hoppe,0.0,0,0.0,0,9.0,1,1.0,0.0,0,11.0
5,Glenna Mcghie,0.0,0,0.0,0,2.0,2,8.0,10.0,1,8.0
6,Gregorio Darr,0.0,0,0.0,0,2.0,2,8.0,9.0,1,9.0
7,Heriberto Martini,0.0,0,0.0,0,2.0,1,8.0,9.0,1,9.0
8,OWNER_TBD,0.0,0,0.0,0,0.0,0,NaN,0.0,0,0.0


In [57]:
test_multipleSprints_EXPECTED['start_committed']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,3.0,2,7.0,9.0,1,8.0
1,Beau Hockensmith,0.0,0,0.0,0,3.0,1,7.0,9.0,1,8.0
2,Bruno Studley,0.0,0,0.0,0,2.0,2,8.0,0.0,0,18.0
3,Craig Garlitz,0.0,0,0.0,0,3.0,1,7.0,10.0,1,7.0
4,Francisco Hoppe,0.0,0,0.0,0,9.0,1,1.0,0.0,0,11.0
5,Glenna Mcghie,0.0,0,0.0,0,2.0,2,8.0,10.0,1,8.0
6,Gregorio Darr,0.0,0,0.0,0,2.0,2,8.0,9.0,1,9.0
7,Heriberto Martini,0.0,0,0.0,0,2.0,1,8.0,9.0,1,9.0
8,OWNER_TBD,0.0,0,0.0,0,0.0,0,NaN,0.0,0,0.0


In [58]:
test_multipleSprints_ACTUAL['start_tasks']

,Owner,Task Type,User Story Id,Planned for Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved
0,Anton Easterday,UNFINISHED_STORIES,UserStory #112,13,2.0,CURRENT_SPRINT,0.0,2.0,0.0
1,Anton Easterday,UNFINISHED_STORIES,UserStory #118,13,1.0,CURRENT_SPRINT,0.0,1.0,0.0
2,Anton Easterday,UNFINISHED_STORIES,UserStory #101,15,9.0,NEXT_SPRINT,0.0,9.0,0.0
3,Beau Hockensmith,UNFINISHED_STORIES,UserStory #170,14,3.0,CURRENT_SPRINT,0.0,3.0,0.0
4,Beau Hockensmith,UNFINISHED_STORIES,UserStory #125,15,9.0,NEXT_SPRINT,0.0,9.0,0.0
5,Bruno Studley,UNFINISHED_STORIES,UserStory #21,13,1.0,CURRENT_SPRINT,0.0,1.0,0.0
6,Bruno Studley,UNFINISHED_STORIES,UserStory #158,12,1.0,CURRENT_SPRINT,0.0,1.0,0.0
7,Craig Garlitz,UNFINISHED_STORIES,UserStory #131,14,3.0,CURRENT_SPRINT,0.0,3.0,0.0
8,Craig Garlitz,UNFINISHED_STORIES,UserStory #36,15,10.0,NEXT_SPRINT,0.0,10.0,0.0
9,Francisco Hoppe,UNFINISHED_STORIES,UserStory #10,14,9.0,CURRENT_SPRINT,0.0,9.0,0.0


In [59]:
test_multipleSprints_EXPECTED['start_tasks']

,Owner,Task Type,User Story Id,Planned for Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved
0,Anton Easterday,UNFINISHED_STORIES,UserStory #112,13,2.0,CURRENT_SPRINT,0.0,2.0,0.0
1,Anton Easterday,UNFINISHED_STORIES,UserStory #118,13,1.0,CURRENT_SPRINT,0.0,1.0,0.0
2,Anton Easterday,UNFINISHED_STORIES,UserStory #101,15,9.0,NEXT_SPRINT,0.0,9.0,0.0
3,Beau Hockensmith,UNFINISHED_STORIES,UserStory #170,14,3.0,CURRENT_SPRINT,0.0,3.0,0.0
4,Beau Hockensmith,UNFINISHED_STORIES,UserStory #125,15,9.0,NEXT_SPRINT,0.0,9.0,0.0
5,Bruno Studley,UNFINISHED_STORIES,UserStory #21,13,1.0,CURRENT_SPRINT,0.0,1.0,0.0
6,Bruno Studley,UNFINISHED_STORIES,UserStory #158,12,1.0,CURRENT_SPRINT,0.0,1.0,0.0
7,Craig Garlitz,UNFINISHED_STORIES,UserStory #131,14,3.0,CURRENT_SPRINT,0.0,3.0,0.0
8,Craig Garlitz,UNFINISHED_STORIES,UserStory #36,15,10.0,NEXT_SPRINT,0.0,10.0,0.0
9,Francisco Hoppe,UNFINISHED_STORIES,UserStory #10,14,9.0,CURRENT_SPRINT,0.0,9.0,0.0


In [60]:
test_multipleSprints_ACTUAL['end_committed']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,0.0,2,0.0,4.0,1,-4.0
1,Beau Hockensmith,0.0,0,0.0,0,0.0,1,0.0,4.0,1,-4.0
2,Bruno Studley,0.0,0,0.0,0,0.0,2,0.0,0.0,0,0.0
3,Craig Garlitz,0.0,0,0.0,0,0.0,1,0.0,5.0,1,-5.0
4,Francisco Hoppe,0.0,0,0.0,0,1.0,1,-1.0,0.0,0,-1.0
5,Glenna Mcghie,0.0,0,0.0,0,0.0,2,0.0,4.0,1,-4.0
6,Gregorio Darr,0.0,0,0.0,0,0.0,2,0.0,3.0,1,-3.0
7,Heriberto Martini,0.0,0,0.0,0,0.0,1,0.0,3.0,1,-3.0
8,OWNER_TBD,0.0,0,0.0,0,0.0,0,NaN,0.0,0,0.0


In [61]:
test_multipleSprints_EXPECTED['end_committed']

,Developer,Rejects (days),Rejects (#),Debugging (days),Debugging (#),Implementation (days),Implementation (#),Bandwidth,NEXT SPRINT (days),NEXT SPRINT (#),NEXT SPRINT Bandwidth
0,Anton Easterday,0.0,0,0.0,0,0.0,2,0.0,4.0,1,-4.0
1,Beau Hockensmith,0.0,0,0.0,0,0.0,1,0.0,4.0,1,-4.0
2,Bruno Studley,0.0,0,0.0,0,0.0,2,0.0,0.0,0,0.0
3,Craig Garlitz,0.0,0,0.0,0,0.0,1,0.0,5.0,1,-5.0
4,Francisco Hoppe,0.0,0,0.0,0,1.0,1,-1.0,0.0,0,-1.0
5,Glenna Mcghie,0.0,0,0.0,0,0.0,2,0.0,4.0,1,-4.0
6,Gregorio Darr,0.0,0,0.0,0,0.0,2,0.0,3.0,1,-3.0
7,Heriberto Martini,0.0,0,0.0,0,0.0,1,0.0,3.0,1,-3.0
8,OWNER_TBD,0.0,0,0.0,0,0.0,0,NaN,0.0,0,0.0


In [62]:
test_multipleSprints_ACTUAL['end_tasks']

,Owner,Task Type,User Story Id,Planned for Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved
0,Anton Easterday,UNFINISHED_STORIES,UserStory #112,13,2.0,CURRENT_SPRINT,2.50,0.0,1.000000
1,Anton Easterday,UNFINISHED_STORIES,UserStory #118,13,1.0,CURRENT_SPRINT,1.25,0.0,1.000000
2,Anton Easterday,UNFINISHED_STORIES,UserStory #101,15,9.0,NEXT_SPRINT,6.25,4.0,0.555556
3,Beau Hockensmith,UNFINISHED_STORIES,UserStory #170,14,3.0,CURRENT_SPRINT,3.75,0.0,1.000000
4,Beau Hockensmith,UNFINISHED_STORIES,UserStory #125,15,9.0,NEXT_SPRINT,6.25,4.0,0.555556
5,Bruno Studley,UNFINISHED_STORIES,UserStory #21,13,1.0,CURRENT_SPRINT,1.25,0.0,1.000000
6,Bruno Studley,UNFINISHED_STORIES,UserStory #158,12,1.0,CURRENT_SPRINT,1.25,0.0,1.000000
7,Craig Garlitz,UNFINISHED_STORIES,UserStory #131,14,3.0,CURRENT_SPRINT,3.75,0.0,1.000000
8,Craig Garlitz,UNFINISHED_STORIES,UserStory #36,15,10.0,NEXT_SPRINT,6.25,5.0,0.500000
9,Francisco Hoppe,UNFINISHED_STORIES,UserStory #10,14,9.0,CURRENT_SPRINT,10.00,1.0,0.888889


In [63]:
test_multipleSprints_EXPECTED['end_tasks']

,Owner,Task Type,User Story Id,Planned for Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved
0,Anton Easterday,UNFINISHED_STORIES,UserStory #112,13,2.0,CURRENT_SPRINT,2.50,0.0,1.000000
1,Anton Easterday,UNFINISHED_STORIES,UserStory #118,13,1.0,CURRENT_SPRINT,1.25,0.0,1.000000
2,Anton Easterday,UNFINISHED_STORIES,UserStory #101,15,9.0,NEXT_SPRINT,6.25,4.0,0.555556
3,Beau Hockensmith,UNFINISHED_STORIES,UserStory #170,14,3.0,CURRENT_SPRINT,3.75,0.0,1.000000
4,Beau Hockensmith,UNFINISHED_STORIES,UserStory #125,15,9.0,NEXT_SPRINT,6.25,4.0,0.555556
5,Bruno Studley,UNFINISHED_STORIES,UserStory #21,13,1.0,CURRENT_SPRINT,1.25,0.0,1.000000
6,Bruno Studley,UNFINISHED_STORIES,UserStory #158,12,1.0,CURRENT_SPRINT,1.25,0.0,1.000000
7,Craig Garlitz,UNFINISHED_STORIES,UserStory #131,14,3.0,CURRENT_SPRINT,3.75,0.0,1.000000
8,Craig Garlitz,UNFINISHED_STORIES,UserStory #36,15,10.0,NEXT_SPRINT,6.25,5.0,0.500000
9,Francisco Hoppe,UNFINISHED_STORIES,UserStory #10,14,9.0,CURRENT_SPRINT,10.00,1.0,0.888889


<h1>test_releaseCycle</h1>

In [100]:
# Implement test logic, and run it

# Test logic
def test_releaseCycle():
    output = {}
    timecard.RANDOM.reset(271) # Set seed so output is the same even if logic invokes random methods
    RELEASE_DURATION = 150
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = initTestData(DEV_DF, PM_DF, 125)
    
    NUMBER_OF_SPRINTS = 25
    SPRINT_DURATION = 10
    entries_df, worksheets = timecard.runReleaseCycle(teamsRepo, ticketsRepo, storiesRepo, datetime(2018, 1, 15), \
                                                   SPRINT_DURATION, NUMBER_OF_SPRINTS)    
    
    anton_df = entries_df[entries_df['Developer'] == 'Anton Easterday']
    glenna_df = entries_df[entries_df['Developer'] == 'Glenna Mcghie']
        
    burnout_df = timecard.releaseBurnout(entries_df)
    
    output['Entries'] = entries_df
    output['Anton'] = anton_df
    output['Glenna'] = glenna_df
    output['Burnout'] = burnout_df

    return output

# Run the test
test_releaseCycle_ACTUAL = test_releaseCycle()

In [101]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_releaseCycle_EXPECTED():
    createExpectedOutput(test_releaseCycle_ACTUAL['Entries'],    'test_releaseCycle.Entries')
    createExpectedOutput(test_releaseCycle_ACTUAL['Anton'],      'test_releaseCycle.Anton')
    createExpectedOutput(test_releaseCycle_ACTUAL['Glenna'],     'test_releaseCycle.Glenna')
    createExpectedOutput(test_releaseCycle_ACTUAL['Burnout'],    'test_releaseCycle.Burnout')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_releaseCycle_EXPECTED()

In [103]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_releaseCycle_EXPECTED = {}

test_releaseCycle_EXPECTED['Entries']      = loadExpectedOutput('test_releaseCycle.Entries')
test_releaseCycle_EXPECTED['Anton']        = loadExpectedOutput('test_releaseCycle.Anton')
test_releaseCycle_EXPECTED['Glenna']       = loadExpectedOutput('test_releaseCycle.Glenna')
test_releaseCycle_EXPECTED['Burnout']      = loadExpectedOutput('test_releaseCycle.Burnout')

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for sensitive fields (any float)
sensitive_fields = ['Time Spent']
testlets = ['Entries', 'Anton', 'Glenna']
for field in sensitive_fields:
    for testlet in testlets:
        rounded = test_releaseCycle_EXPECTED[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_releaseCycle_EXPECTED[testlet][field] = rounded
        rounded = test_releaseCycle_ACTUAL[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_releaseCycle_ACTUAL[testlet][field] = rounded
sensitive_fields = ['Effort']
testlets = ['Burnout']
for field in sensitive_fields:
    for testlet in testlets:
        rounded = test_releaseCycle_EXPECTED[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_releaseCycle_EXPECTED[testlet][field] = rounded
        rounded = test_releaseCycle_ACTUAL[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_releaseCycle_ACTUAL[testlet][field] = rounded


EXPECTED['test_releaseCycle.Entries']        = test_releaseCycle_EXPECTED['Entries']
EXPECTED['test_releaseCycle.Anton']          = test_releaseCycle_EXPECTED['Anton']
EXPECTED['test_releaseCycle.Glenna']         = test_releaseCycle_EXPECTED['Glenna']
EXPECTED['test_releaseCycle.Burnout']        = test_releaseCycle_EXPECTED['Burnout']

ACTUAL['test_releaseCycle.Entries']          = test_releaseCycle_ACTUAL['Entries']
ACTUAL['test_releaseCycle.Anton']            = test_releaseCycle_ACTUAL['Anton']
ACTUAL['test_releaseCycle.Glenna']           = test_releaseCycle_ACTUAL['Glenna']
ACTUAL['test_releaseCycle.Burnout']          = test_releaseCycle_ACTUAL['Burnout']

testOK('test_releaseCycle.Entries'), \
testOK('test_releaseCycle.Anton'), \
testOK('test_releaseCycle.Glenna'), \
testOK('test_releaseCycle.Burnout'), \

C:\Alex\CodeImages\technos\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(False, False, False, True)

In [104]:
test_releaseCycle_ACTUAL['Entries']

,Sprint,Date,Team,Developer,User Story,Task Type,Time Spent
0,1,2018-01-29,Team A,Anton Easterday,UserStory #151,UNFINISHED_STORIES,2.50
1,1,2018-01-29,Team A,Anton Easterday,UserStory #97,UNFINISHED_STORIES,2.50
2,1,2018-01-29,Team A,Anton Easterday,UserStory #58,UNFINISHED_STORIES,5.00
3,1,2018-01-29,Team A,Anton Easterday,UserStory #83,UNFINISHED_STORIES,0.00
4,1,2018-01-29,Team A,Beau Hockensmith,UserStory #1,UNFINISHED_STORIES,10.00
5,1,2018-01-29,Team A,Bruno Studley,UserStory #77,UNFINISHED_STORIES,3.75
6,1,2018-01-29,Team A,Bruno Studley,UserStory #137,UNFINISHED_STORIES,2.50
7,1,2018-01-29,Team A,Bruno Studley,UserStory #52,UNFINISHED_STORIES,3.75
8,1,2018-01-29,Team A,Craig Garlitz,UserStory #143,UNFINISHED_STORIES,10.00
9,1,2018-01-29,Team A,Craig Garlitz,UserStory #75,UNFINISHED_STORIES,0.00


In [105]:
test_releaseCycle_EXPECTED['Entries']

,Sprint,Date,Team,Developer,User Story,Task Type,Time Spent
0,1,2018-01-29,Team A,Anton Easterday,UserStory #151,UNFINISHED_STORIES,2.50
1,1,2018-01-29,Team A,Anton Easterday,UserStory #97,UNFINISHED_STORIES,2.50
2,1,2018-01-29,Team A,Anton Easterday,UserStory #58,UNFINISHED_STORIES,5.00
3,1,2018-01-29,Team A,Anton Easterday,UserStory #83,UNFINISHED_STORIES,0.00
4,1,2018-01-29,Team A,Beau Hockensmith,UserStory #1,UNFINISHED_STORIES,10.00
5,1,2018-01-29,Team A,Bruno Studley,UserStory #77,UNFINISHED_STORIES,3.75
6,1,2018-01-29,Team A,Bruno Studley,UserStory #137,UNFINISHED_STORIES,2.50
7,1,2018-01-29,Team A,Bruno Studley,UserStory #52,UNFINISHED_STORIES,3.75
8,1,2018-01-29,Team A,Craig Garlitz,UserStory #143,UNFINISHED_STORIES,10.00
9,1,2018-01-29,Team A,Craig Garlitz,UserStory #75,UNFINISHED_STORIES,0.00


In [106]:
test_releaseCycle_ACTUAL['Anton']

,Sprint,Date,Team,Developer,User Story,Task Type,Time Spent
0,1,2018-01-29,Team A,Anton Easterday,UserStory #151,UNFINISHED_STORIES,2.50
1,1,2018-01-29,Team A,Anton Easterday,UserStory #97,UNFINISHED_STORIES,2.50
2,1,2018-01-29,Team A,Anton Easterday,UserStory #58,UNFINISHED_STORIES,5.00
3,1,2018-01-29,Team A,Anton Easterday,UserStory #83,UNFINISHED_STORIES,0.00
74,2,2018-02-12,Team A,Anton Easterday,UserStory #83,UNFINISHED_STORIES,2.50
75,2,2018-02-12,Team A,Anton Easterday,UserStory #119,UNFINISHED_STORIES,6.25
76,2,2018-02-12,Team A,Anton Easterday,UserStory #44,UNFINISHED_STORIES,1.25
77,2,2018-02-12,Team A,Anton Easterday,UserStory #55,UNFINISHED_STORIES,0.00
78,2,2018-02-12,Team A,Anton Easterday,UserStory #121,UNFINISHED_STORIES,0.00
191,3,2018-02-26,Team A,Anton Easterday,UserStory #55,UNFINISHED_STORIES,1.25


In [107]:
test_releaseCycle_EXPECTED['Anton']

,Sprint,Date,Team,Developer,User Story,Task Type,Time Spent
0,1,2018-01-29,Team A,Anton Easterday,UserStory #151,UNFINISHED_STORIES,2.50
1,1,2018-01-29,Team A,Anton Easterday,UserStory #97,UNFINISHED_STORIES,2.50
2,1,2018-01-29,Team A,Anton Easterday,UserStory #58,UNFINISHED_STORIES,5.00
3,1,2018-01-29,Team A,Anton Easterday,UserStory #83,UNFINISHED_STORIES,0.00
4,2,2018-02-12,Team A,Anton Easterday,UserStory #83,UNFINISHED_STORIES,2.50
5,2,2018-02-12,Team A,Anton Easterday,UserStory #119,UNFINISHED_STORIES,6.25
6,2,2018-02-12,Team A,Anton Easterday,UserStory #44,UNFINISHED_STORIES,1.25
7,2,2018-02-12,Team A,Anton Easterday,UserStory #55,UNFINISHED_STORIES,0.00
8,2,2018-02-12,Team A,Anton Easterday,UserStory #121,UNFINISHED_STORIES,0.00
9,3,2018-02-26,Team A,Anton Easterday,UserStory #55,UNFINISHED_STORIES,1.25


In [108]:
test_releaseCycle_ACTUAL['Glenna']

,Sprint,Date,Team,Developer,User Story,Task Type,Time Spent
12,1,2018-01-29,Team A,Glenna Mcghie,UserStory #164,UNFINISHED_STORIES,7.50
13,1,2018-01-29,Team A,Glenna Mcghie,UserStory #33,UNFINISHED_STORIES,2.50
93,2,2018-02-12,Team A,Glenna Mcghie,UserStory #33,UNFINISHED_STORIES,2.50
94,2,2018-02-12,Team A,Glenna Mcghie,UserStory #11,UNFINISHED_STORIES,3.75
95,2,2018-02-12,Team A,Glenna Mcghie,UserStory #59,UNFINISHED_STORIES,3.75
96,2,2018-02-12,Team A,Glenna Mcghie,UserStory #73,UNFINISHED_STORIES,0.00
207,3,2018-02-26,Team A,Glenna Mcghie,UserStory #59,UNFINISHED_STORIES,1.25
208,3,2018-02-26,Team A,Glenna Mcghie,UserStory #73,UNFINISHED_STORIES,1.25
209,3,2018-02-26,Team A,Glenna Mcghie,UserStory #5,UNFINISHED_STORIES,3.75
210,3,2018-02-26,Team A,Glenna Mcghie,UserStory #69,UNFINISHED_STORIES,3.75


In [112]:
test_releaseCycle_EXPECTED['Glenna']

,Sprint,Date,Team,Developer,User Story,Task Type,Time Spent
0,1,2018-01-29,Team A,Glenna Mcghie,UserStory #164,UNFINISHED_STORIES,7.50
1,1,2018-01-29,Team A,Glenna Mcghie,UserStory #33,UNFINISHED_STORIES,2.50
2,2,2018-02-12,Team A,Glenna Mcghie,UserStory #33,UNFINISHED_STORIES,2.50
3,2,2018-02-12,Team A,Glenna Mcghie,UserStory #11,UNFINISHED_STORIES,3.75
4,2,2018-02-12,Team A,Glenna Mcghie,UserStory #59,UNFINISHED_STORIES,3.75
5,2,2018-02-12,Team A,Glenna Mcghie,UserStory #73,UNFINISHED_STORIES,0.00
6,3,2018-02-26,Team A,Glenna Mcghie,UserStory #59,UNFINISHED_STORIES,1.25
7,3,2018-02-26,Team A,Glenna Mcghie,UserStory #73,UNFINISHED_STORIES,1.25
8,3,2018-02-26,Team A,Glenna Mcghie,UserStory #5,UNFINISHED_STORIES,3.75
9,3,2018-02-26,Team A,Glenna Mcghie,UserStory #69,UNFINISHED_STORIES,3.75


In [110]:
test_releaseCycle_ACTUAL['Burnout']

,Sprint,Stories Started,Stories Progressed,Stories Completed,Effort,Active Developers
0,1,74,74,32,300.00,30
1,2,75,117,78,300.00,30
2,3,65,104,60,300.00,30
3,4,63,107,60,300.00,30
4,5,43,90,56,300.00,30
5,6,53,87,49,300.00,30
6,7,46,84,47,300.00,30
7,8,35,72,32,300.00,30
8,9,39,79,39,300.00,30
9,10,34,74,37,300.00,30


In [111]:
test_releaseCycle_EXPECTED['Burnout']

,Sprint,Stories Started,Stories Progressed,Stories Completed,Effort,Active Developers
0,1,74,74,32,300.00,30
1,2,75,117,78,300.00,30
2,3,65,104,60,300.00,30
3,4,63,107,60,300.00,30
4,5,43,90,56,300.00,30
5,6,53,87,49,300.00,30
6,7,46,84,47,300.00,30
7,8,35,72,32,300.00,30
8,9,39,79,39,300.00,30
9,10,34,74,37,300.00,30


<h1>test_FOO: template for tests that produce a DataFrame output </h1>
<li> Copy to create a new test, and replace 'FOO' by the name of your test.
<li> Implement test logic in test_FOO()
<li> To create expected output once test logic is producing right output, temporarily uncomment the call to createExpectedOutput, call it, and comment it again. This saves expected output as a CSV file so subsequent runs of the test can load it and verify that the test still produces the same output as before.
<li> Uncomment lines that load expected output and remove the dummy line next to it creating a dummy expected output.
<li> Display the outputs for convenience in debugging when test fails.

In [71]:
# Implement test logic, and run it

#Test logic
def test_FOO():
    output = {}
    timecard.RANDOM.reset(271) # Set seed so output is the same even if logic invokes random methods
    teams_df, stories_df, teamsRepo, storiesRepo, ticketsRepo = initTestData(DEV_DF, PM_DF, 125)
    
    dummyA_df = pd.DataFrame({'A': [45.123456789, 34.123456789]})
    dummyB_df = pd.DataFrame({'B': [2333.123456789, 3984.123456789]})
    
    output['Dummy_A'] = dummyA_df
    output['Dummy_B'] = dummyB_df

    return output

# Run the test
test_FOO_ACTUAL = test_FOO()

In [72]:
# Uncomment to update expected output to match the actual one

# Helper method
def create_FOO_EXPECTED():
    createExpectedOutput(test_FOO_ACTUAL['Dummy_A'],    'test_FOO.Dummy_A')
    createExpectedOutput(test_FOO_ACTUAL['Dummy_B'],    'test_FOO.Dummy_B')

# Uncomment to update expected output to match the actual one, and then put the comment back
#create_FOO_EXPECTED()

In [73]:
# Load expected output, update the EXPECTED and ACTUAL dictionaries, and check test is OK
test_FOO_EXPECTED = {}

test_FOO_EXPECTED['Dummy_A']      = loadExpectedOutput('test_FOO.Dummy_A')
test_FOO_EXPECTED['Dummy_B']      = loadExpectedOutput('test_FOO.Dummy_B')

# Rounding inaccuracies in saving and loading CSV will create an artificial mismatch between ACTUAL and EXPECTED
# So round EXPECTED and ACTUAL to 6 decimal places for sensitive fields (any float)
sensitive_fields = ['A']
testlets = ['Dummy_A']
for field in sensitive_fields:
    for testlet in testlets:
        rounded = test_FOO_EXPECTED[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_FOO_EXPECTED[testlet][field] = rounded
        rounded = test_FOO_ACTUAL[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_FOO_ACTUAL[testlet][field] = rounded
sensitive_fields = ['B']
testlets = ['Dummy_B']
for field in sensitive_fields:
    for testlet in testlets:
        rounded = test_FOO_EXPECTED[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_FOO_EXPECTED[testlet][field] = rounded
        rounded = test_FOO_ACTUAL[testlet][field].apply(lambda x: round(x, 6)) # Round to 6 decimal places
        test_FOO_ACTUAL[testlet][field] = rounded


EXPECTED['test_FOO.Dummy_A']        = test_FOO_EXPECTED['Dummy_A']
EXPECTED['test_FOO.Dummy_B']        = test_FOO_EXPECTED['Dummy_B']

ACTUAL['test_FOO.Dummy_A']          = test_FOO_ACTUAL['Dummy_A']
ACTUAL['test_FOO.Dummy_B']          = test_FOO_ACTUAL['Dummy_B']

testOK('test_FOO.Dummy_A'), \
testOK('test_FOO.Dummy_B'), \

(True, True)

In [74]:
test_FOO_ACTUAL['Dummy_A']

,A
0,45.123457
1,34.123457


In [75]:
test_FOO_EXPECTED['Dummy_A']

,A
0,45.123457
1,34.123457


In [76]:
test_FOO_ACTUAL['Dummy_B']

,B
0,2333.123457
1,3984.123457


In [77]:
test_FOO_EXPECTED['Dummy_B']

,B
0,2333.123457
1,3984.123457


<h1>Run all the tests in this notebook</h1>

In [78]:
# Uncomment to run all the tests defined in this notebook
#unittest.main(argv=[''], verbosity=2, exit=False)